# q3dfit example notebook: rest frame optical, Gemini+GMOS single spectrum of ORC2/3C from [Rupke et al. 2024](https://ui.adsabs.harvard.edu/abs/2024ApJ...967...51R/abstract)

<h3><font color='teal'>Installation of the environment and the package are described <a href="https://q3dfit.readthedocs.io/">here</a>. </font></h3>

This Jupyter notebook allows you to run Q3Dfit, a PSF decomposition and spectral analysis package tailored for JWST NIRSpec and MIRI IFU observations. 

Q3Dfit is developed as a science-enabling data product by the Early Release Science Team #1335 Q3D. You can find more information about this ERS program **Q3D** [here](https://q3d.github.io/) and [here](https://www.stsci.edu/jwst/science-execution/approved-programs/dd-ers/program-1335).

The software is based on the existing package IFSFIT developed by Dave Rupke (see [ADS link](https://ui.adsabs.harvard.edu/abs/2017ApJ...850...40R/abstract)).

The following notebook will guide you through the initialization procedure and will then perform the analysis. 

## 1. Initialization

In [ ]:
import os.path
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
# Be sure to set the path to q3dfit correctly.
# For instance:
#import sys
#sys.path.append('/Users/jwstuser/q3dfit/')
#import sys
#sys.path.append("../")

### 1.0. Setting up the directory tree

Define the directories in which the data that you want to analyse are stored and the output directories. We recommend creating a working directory that you name after your target, in which all outputs from q3dfit will be saved. Then download test data.

In [ ]:
# Base directory (book-keeping)
volume = 'orc23c/'
# prefix label for output files
label = 'orc23c'
# Input directory
indir = volume
if not os.path.exists(indir):
    os.makedirs(indir)
# Output directory
outdir = volume
if not os.path.exists(outdir):
    os.makedirs(outdir)
# Initialization file (q3di.npy) directory
initdir = volume
# Output logfile
logfile = os.path.join(outdir, label+'-fitlog.txt')

Download data from public Box folder:

In [ ]:
# make tuples of urls and download filenames
# stellartemplates = stellar population synthesis templates
infile_tup=('https://rhodes.box.com/shared/static/1k743ttc75vd0aoal4w7j6bhyvcgbz44.fits','orc23c.fits')
stellartemplates_tup1=('https://rhodes.box.com/shared/static/tqnvr2g3xcs6wedpoa0yhtshvivk1wdi.npy','ckc14_solarZ_res30kms_2000to7000A.npy')
stellartemplates_tup2=('https://rhodes.box.com/shared/static/xuif7n0g5qxmgrdt5xn3fje92v0hm8kv.npy','bpass_v2.3.a+00_z001to040_bin_lam2400to10000.npy')
# download files; by default don't force overwrite and take first element of output
from q3dfit.nonfit.jnb import download_files
infile = download_files(infile_tup, indir, force=False)[0]
stellartemplates1 = download_files(stellartemplates_tup1, indir, force=False)[0]
stellartemplates2 = download_files(stellartemplates_tup2, indir, force=False)[0]
# add subdirectory to filenames
infile = os.path.join(indir, infile)
stellartemplates1 = os.path.join(indir, stellartemplates1)
stellartemplates2 = os.path.join(indir, stellartemplates2)

### 1.1. Initializing the fit

The initial parameters of the fit are stored in an object of class `q3din`. Each parameter or attribute of this class controls some aspect of the fit process. We start by instantiating the class. The only required parameters at the outset are the input data and label; the label is used for output file naming. 

In [ ]:
from q3dfit.q3din import q3din
q3di = q3din(infile, label, outdir=outdir, logfile=logfile, dqext=None)

Here's a list of the fit parameters that are automatically set:

In [ ]:
q3di.__dict__

### 1.2. Setting up the data and models

Some general information about your data. `argsreadcube` is a dictionary of attributes sent to the `Cube` class.
- For non-JWST data, set `wmapext` to `None`. The WMAP extension is a [3-D weight image](https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/science_products.html) giving the relative weights of the output spaxels.
- Microns are the wavelength unit used internally, but `q3dfit` can accept input/output in Å.
- `q3dit` does calculations in f$_\lambda$ space, but assumes input units of MJy/sr, the JWST default. Other input wavelength units can be specified. In this case, the data is in erg/s/cm$^2$/Å. The output flux units will be the same. We also normalize to minimize numerical issues in the fitting.

In [ ]:
q3di.argsreadcube = {'wmapext': None,
                     'waveunit_in': 'Angstrom',
                     'fluxunit_in': 'erg/s/cm2/Angstrom',
                     'waveunit_out': 'Angstrom',
                     'fluxunit_out': 'erg/s/cm2/Angstrom',
                     'fluxnorm': 1e-17}
cube = q3di.load_cube()

Let's plot the spectrum to see how it looks. The arguments are column and row in unity-offset integers; since this is a single spectrum, we specify 1 for both. The flux units are as specified above.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=[10,4])
spec_test = cube.specextract(1, 1, ylim=[0.,10.])

Name and systemic redshift of the galaxy. `zsys_gas` is used in this notebook in initializing the arrays of initial guesses below.

In [ ]:
q3di.name = 'ORC23C'
q3di.zsys_gas = 0.2521

Wavelength range over which to fit data. The user can also specify sets of regions to ignore in the fit.

In [ ]:
q3di.fitrange = [4735.,10335.]

### 1.3. Setting up the fitting parameters

#### 1.3.1. Emission-line parameters

What lines do you want to fit? You can choose from the linelists available [here](https://github.com/Q3D/q3dfit/tree/main/q3dfit/data/linelists), or in `q3dfit/data/linelists/`.

In [ ]:
lines = ['Halpha', 'Hbeta', '[OI]6300', '[OI]6364', '[OIII]4959', '[OIII]5007', '[NII]6548', 
         '[NII]6583', '[SII]6716', '[SII]6731', '[ArIII]7138','[ArIII]7753']

This block sets up initial conditions for the emission-line fit. This initialization method adds a number of new attributes to the object. Emission lines are set to a common redshift and velocity dispersion, set to `q3di.zsys_gas` and 50 km/s by default. However, different sets of emission lines can have different velocities and linewidths by specifying different lines to which to tie particular emission lines. Different initial conditions can also be set on a line-by-line basis.

The default number of velocity components is 1. 

`siglim_gas` sets lower and upper bounds for the Gaussian width (sigma) of the emission line. These limits can be set for all components by defining a 2-element array when initializing the fit. The limits can also be set for individual components later by defining an (1 x 1 x Ncomp x 2) array, or by modifying the initial array using slicing.

In [ ]:
q3di.init_linefit(lines, linetie='Halpha', siglim_gas=np.array([1., 1500.]))
q3di.__dict__.keys()

Let's see what one of these new attributes looks like, just for fun:

In [ ]:
q3di.zinit_gas

##### Line ratio constraints
Lines with ratios fixed by atomic physics have their ratios fixed automatically. Other ratios can have bound constraints applied, or they can be fixed to a particular value. Some density-sensitive ratios have bounds applied; these limits can be found in [this table](/q3dfit/data/linelists/doublets.tbl).

See [the documentation](https://q3dfit.readthedocs.io/stable/q3dfit.html#q3dfit.lineinit.lineinit) for more details.

- `line1`, `line2`, and `comp` are required. `comp` is an array of velocity components (zero-indexed) on which to apply the constraints, one array for each pair of lines.
- `value` is the initial value of `line1`/`line2`. Presently, if `value` is specified for one pair of lines, it must be specified for all. Otherwise, the initial value is determined from the data.
- The ratio can be `fixed` to the initial value. Presently, if `fixed` is defined, it must be set to `True` or `False` for all pairs of line.
- If the ratio is not `fixed`, `lower` and `upper` limits can also be specified. (If they are not, and the line pair is a doublet in the doublets.tbl file, then the lower and upper limits are set using the data in that file.) Presently, if `lower` or `upper` is defined here for one set of lines, it must be defined here for every pair of lines.

In [ ]:
#q3di.argslineinit = dict()

# Required columns:
#line1 = ['[NI]5200']
#line2 = ['[NI]5198']
#comp = np.array([[0]], dtype=np.int32)
# Optional columns:
#value = [2./3.]
#fixed = [True]

# Write table
#from astropy.table import QTable
#lineratio = QTable([line1, line2, comp, value, fixed], names=['line1', 'line2', 'comp', 'value', 'fixed'])
#q3di.argslineinit['lineratio']=lineratio

##### Spectral resolution convolution

If no convolution is desired: do not set `spect_convol` (or set it to `{}`, or `None`).

If convolution is desired: `spect_convol` is a dictionary with two optional tags.
- `ws_instrum`: This specifies the desired convolution method. The syntax is: `{INSTRUMENT:[GRATING]}`. The values for `INSTRUMENT` and `GRATING` for pre-defined dispersion files should mirror the filename syntax in `q3dfit/data/dispersion_files/`. E.g., for file `jwst_miri_ch1a_disp.fits`, `INSTRUMENT=jwst_miri` and `GRATING=ch1a`. (Case is irrelevant. For convolution with a constant value of spectral resolution [R], Δλ FWHM in [$\mu$m], or velocity in [km/s], set `INSTRUMENT = flat` and `GRATING = ` a string containing `R`, `dlam`, or `dvel` and the corresponding numerical quantity. More thana one instrument and/or grating can be set.
- `dispdir`: Directory in which to find the dispersion files. If not set, the default `q3dfit` directory is searched.

Examples: 
1. flat R=500: `spect_instrum = {'flat':['R500']}`
2. flat velocity FWHM = 30km/s: `spect_instrum = {'flat':['dvel30']}`
3. flat Δλ FWHM = 4 Å: `spect_instrum = {'flat':['dlambda0.0004']}`
4. JWST NIRSPEC / G140M: `spect_instrum = {'JWST_NIRSPEC':['G140M']}`
5. Spitzer IRS SH+LH: `spect_instrum = {'Spitzer_IRS':['ch1_sh','ch1_lh']}`

Note in the final example that two gratings are set.

This case applies to the GMOS B600 grating. Gemini website says R = 1918 at 7640 A for 0.5" slit. This gives 5.97 A FWHM for our 0.75" slit.

In [ ]:
q3di.spect_convol['ws_instrum'] = {'flat':['dlambda5.97']}

##### Creating convolution files (optional)

To create a dispersion file, use one of the following methods. The second two involve specific subclasses of the dispersion class used for the instrument/grating file or constant dispersion formats 

1. Create a `dispersion` object and use the `dispersion.write()` method. For example:

```
dispEx1 = dispersion()
dispEx1.write('/dispdir/disp.fits', wave=np.linspace(5.,10.,50), type='R', disp=np.full(50, 500.))
```

2. Create a `InstGratDispersion` object to attach instrument and grating information to the object and define the output filename in the `q3dfit` format. Use the `InstGratDispersion.writeInstGrat()` method.

```
dispEx2 = InstGratDispersion(`Keck_ESI`,`echellette`, dispdir=`/dispdir/`)
dispEx2.writeInstGrat(wave=np.linspace(5.,10.,50), type='dvel', disp=np.full(50, 30.))
```

3. Create a `FlatDispersion` object and use the `FlatDispersion.writeFlat()` method. This requires only a single value for the dispersion quantity and also defines the filename automatically.
```
dispEx3 = FlatDispersion(0.0004,`dlam`,wave=np.linspace(5.,10.,50))
dispEx3.writeFlat(dispdir=`/dispdir/`)
```

##### Options to `lmfit` and `scipy.optimize.least_squares` (or `scipy.optimize.leastsq`, or other optimizer)
`q3dfit` uses the `fit` method of the [`Model` class](https://lmfit.github.io/lmfit-py/model.html#lmfit.model.Model) of `lmfit` to call [`scipy.optimize.least_squares`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html), or another method of choice. Both the method and function have options which can be changed in the `q3dfit` call. To do so, add key/value pairs to the `argslinefit` dictionary, which in turn is a keyword of the `q3di` dictionary.

The options to the `fit` method in `lmfit` that can currently be changed are the following:
- `method`: in particular, `leastsq` may be faster than `least_squares`
- `max_nfev`: maximum number of function evaluations before the fit aborts
- `iter_cb`: if this is set to "per_iteration", the value of every model parameter at each function evaluation is printed to `stdout`

Most parameters of `least_squares`/`leastsq` can be changed in this way, unless they are specifically set by `lmfit`. Examples which have been tested include:
- `x_scale`: jac
- `tr_solver`: lsmr
- `loss`: soft_l1
- `ftol`, `gtol`, `xtol`
- `epsfcn`

In [ ]:
#q3di.argslinefit['method'] = 'leastsq'
#q3di.argslinefit['iter_cb'] = 'per_iteration'
# As an example, to change the criteria for fit convergence from the defaults of 1.e-8 to 1.e-10:
q3di.argslinefit['ftol'] = 1.e-10
q3di.argslinefit['gtol'] = 1.e-10
q3di.argslinefit['xtol'] = 1.e-10
q3di.argslinefit['x_scale'] = 'jac'
q3di.argslinefit['tr_solver'] = 'lsmr'
# .. and the "suitable step length for the forward- difference approximation of the Jacobian. Normally the actual step length will be sqrt(epsfcn)*x"
# this is only for scipy.optimize.leastsq
#q3di.argslinefit['epsfcn'] = 1.e-15

#### 1.3.2 Continuum parameters

We next initialize the continuum. As part of this, we give it the name of our continuum fitting function, and then add some stellar templates to fit. There are two stellar template libraries to try. One is ...

In [ ]:
from q3dfit import templates

templatefile = indir+'bpass-v2.3.a+00-2000to10000A-z008to040.npy'
templates.read_bpass('/Users/drupke/Documents/stellar_models/bpass/bpass_v2.3.a+00/',
                     templatefile,
                     waverange=[2000.,10000.],
                     zs=[0.008,0.010,0.014,0.020,0.030,0.040])

In [ ]:
q3di.init_contfit('ppxf')
#q3di.startempfile = stellartemplates1
q3di.startempfile = templatefile
q3di.startempwaveunit = 'Angstrom'
q3di.__dict__.keys()

`q3dfit` first masks emission lines before fitting. The default mask value is 500 km/s for each velocity component for the first fit. During the second fit, the mask value is set automatically using the best-fit linewidths determined from the first fit.

In [ ]:
q3di.maskwidths_def

The continuum fitting parameters specified here are for a `ppxf` fit. These are presently the only options available.
- `add_poly_degree` = [Optional] additive Legendre polynomial for host [starlight] component. -1 means no polynomial. Default is 4.
- `av_star` = [Optional] initial guess for Calzetti (2000) reddening. Default is None, which means no reddening is fit.

In [ ]:
q3di.argscontfit['add_poly_degree'] = -1
q3di.argscontfit['mult_poly_degree'] = 0
q3di.av_star = 0.1

## 2. Run fit

Run the fit. Choose `quiet=False` for verbose output. An output dictionary for each spaxel is saved to a numpy binary file labeled with prefix `q3di['label']` and suffix `_col_row.npy`. See note above on multicore processing.

In [ ]:
from q3dfit.q3df import q3dfit
q3dfit(q3di, quiet=False)

## 3. Plot fit results

Load the output of a fit.

In [ ]:
from q3dfit.q3dout import load_q3dout
q3do = load_q3dout(q3di)

Set up the line plot parameters using a dictionary.

* `nx`: Number of subplots in the horizontal direction (default = 1)
* `ny`: Number of subplots in the vertical direction (default = 1)
* Required: choose one options for centerting the plot
    - `line`: a string list of line labels
    - `center_obs`: a float list of wavelengths of each subplot center, in the observed (plotted) frame
    - `center_rest`: a float list of wavelengths of each subplot center, in the rest frame, which are converted to obs. frame
* `size`: float list of widths in wavelength space of each subplot; if not specified (default = 300 $Å$)
* `questfit`: set to `True` to use plot tailored for rest-frame mid-IR spectra

In [ ]:
argsplotline = dict()
argsplotline['nx'] = 3
argsplotline['ny'] = 2
argsplotline['line'] = ['[OIII]4959', '[OI]6300', 'Halpha', '[SII]6716', '[ArIII]7138']
argsplotline['size'] = [300.,200.,150.,100.,100.]
argsplotline['figsize'] = (10,8)

Run the plot method. The output can be saved to a file by specifying `savefig=True`. A default filename is used, which can be overridden by specifying `outfile=file`. The output file will have the suffix `_lin` attached, so that the actual filename will be "file_lin.png". The file format, and other [pyplot.savefig](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html) options, can be added to the `plotargs` dictionary as, e.g., `'argssavefig': {'format': '.pdf'}`.

In [ ]:
q3do.plot_line(q3di, savefig=True, plotargs=argsplotline)

Some line fluxes, in units of fluxnorm:

In [ ]:
q3do.line_fitpars['flux']

In [ ]:
q3do.line_fitpars['z']['Halpha']

Run two methods. The first computes the continuum values to plot, and the second does the plotting. The `plot_cont` method calls the continuum plotting function `plotcont`.

In [ ]:
q3do.sepcontpars(q3di)
q3do.plot_cont(q3di, savefig=True, plotargs={'figsize': (10,7)})

Now print some of the stellar parameter outputs:

In [ ]:
q3do.print_stelpars()